# Introduction

Fichier qui permet d'exporter les differentes donnees necessaires a la diffusion des graphs sur le web, via quelques fonctions qui les retraitent pour correspondre aux filtres attendus.

In [1]:
import gen_dataset_810 as mod_810fr
# importer datetime pour typer les donnees de dates des chaines et des videos
import datetime
import pandas as pd

# Init YouTube Data
datemin = datetime.datetime(2021, 1, 1, 0, 0, 0)
folder_candidats = './data_sources/stats_videos/2022_03_08/'
dt_candidats = mod_810fr.mb_ds()
videos_candidats, chaines_candidats  = dt_candidats.get_full_data_candidat(folder = folder_candidats)
videos_candidats = videos_candidats[videos_candidats['dateVideo'] > datemin]

# Transcript files folder
transcripts_folder = "./data_sources/transcripts_videos/"

# Output folders
folder_json = './data_output/wordCloud-json/'
folder_csv = './data_output/wordCloud-csv/'

Imported 32 channel data


# 1. Fonctions et paramètres globaux

## 1.1. Liste d'exceptions en plus de NLTK default

Liste des mots qu'on va supprimer de nos traitements pour obtenir des "nuages de mots" plus clairs

_N.B : On ne fait pas de nuages parce qu'on n'aime pas trop le style_


In [2]:
# Liste exclusion mot commune
liste_exclusion_additionnelle = ['d', 
                'du', 
                'de', 
                'la', 
                'des', 
                'le', 
                'et', 
                'est', 
                'elle', 
                'une', 
                'en', 
                'que', 
                'aux', 
                'qui', 
                'ces', 
                'les', 
                'dans', 
                'sur', 
                'l', 
                'un', 
                'pour', 
                'par', 
                'il', 
                'ou', 
                'à', 
                'ce', 
                'a', 
                'sont', 
                'cas', 
                'plus', 
                'leur', 
                'se', 
                's', 
                'vous', 
                'au', 
                'c', 
                'aussi', 
                'toutes', 
                'autre', 
                'comme', 
                "c'est",
                "parce",
                "ya",
                "ça", 
                "donc",
                "où",
                "notamment",
                "alors",
                "cette",
                "cet",
                "si", 
                "dis",
                "dit",
                "dire",
                "fait", 
                "faire", 
                "fais",
                "car", 
                "là",
                'non',
                'tous',
                'puis',
                'cela',
                'abord',
                'quand',
                'après',
                'très',
                'entre',
                'tout',
                'voilà',
                'tous',
                'oui',
                'va',
                'ans',
                'veux',
                'question',
                'hui',
                'avoir',
                'eric',
                'emmanuel',
                'yannick',
                'valérie',
                'hui', 'tout', 'leurs', 'dix', 'très',
                'jean', 'luc', 'ceux', 'celles',
                'peut', 'faut', 'cinq', '35', 'veut', 'peux', 'veux',
                'vais', 'bien', 'mal', 'fois',
                'pourquoi', 'comment',
                'beaucoup', 'peu', 'deux',
                'monsieur', 'messieurs', 'madame', 'mesdames',
                'chose', 'être', 'gens',
                'depuis', 'encore',
                'aujourd',
                'autre',
                'quoi',
                'applaudissements'
                ]

## 1.2. Fonctions nécessaires à nos différents traitements

### 1.2.1. Récupération des fichiers de transcripts et transformation de ces derniers en une variable de texte

In [3]:
from pathlib import Path
import re
import ast


# Dump des fichiers associé à chacun des candidats 
def dump_liste_videos_par_candidat(dtsource, liste_candidats = [], periode_de_temps = {}, complement_titre = '') :
    cnt = 0
    for candidat in liste_candidats :
        subdtf_candidat = dtsource[dtsource['candidat'] == candidat]
        subdtf_candidat = subdtf_candidat[subdtf_candidat['dateVideo'] > periode_de_temps['datemin']]
        subdtf_candidat = subdtf_candidat[subdtf_candidat['dateVideo'] < periode_de_temps['datemax']]
        subdtf_candidat.to_csv(dossier_source+nom_fichier+"_"+str(cnt)+".csv") #+"_"+ complement_titre 
        cnt += 1

    return

# Vérifier la présence des transcripts pour les vidéos des candidats
def check_exhau_transcript(dtf_candidat, dossier_ts_associe = './data_sources/transcripts_videos/') : 
    data_manquante = []
    data_presente = []

    dtf_videos_candidat = dtf_candidat
    liste_videos_candidats = dtf_videos_candidat.vid_id.unique()
    for video_id in liste_videos_candidats : 
        fichier_ts = Path(dossier_ts_associe+"/"+ video_id+".json")
        if Path.is_file(fichier_ts) :
            data_presente.append(video_id)
        else :
            # TODO : Dropper aussi le titre de la video pour la retrouver plus facilement
            data_manquante.append(video_id)
    
    print(str(len(data_manquante)) + "/" +str(dtf_candidat.shape[0]) + " n'ont pas de transcript. Peut-être les sous-titres furent-ils désactivés")

    return data_manquante, data_presente

# A partir du nom d'un candidat, on retrouve le nom du dossier 
#   -> via l'expression reguliere utilisée pour créer son dossier à la base (pas de signes diacritiques)
def get_dossier_candidat(nom = '') : 
    clean_name = re.sub(r"[^a-zA-Z0-9]","",nom)
    return clean_name

# Les fichiers de transcripts, même s'ils s'appellent json sont en fait, des fichiers qui contiennent des éléments sous la forme de
# listes de dictionnaires. C'est ce que la fonction retournera pour un fichier.
def get_transcript_as_list(nomfichier = '') : 
    with open(nomfichier) as f:
        data = ast.literal_eval(f.read())
    return data

# Retourne une variable de texte sur la base du fichier de transcript passé en paramètre.
# N.B : Tous les passages à la ligne sont transformés en espaces
def create_text_variable_for_transcript(nomfichier = '') :
    liste_data = get_transcript_as_list(nomfichier=nomfichier)
    text_result = ''
    for i in range(len(liste_data)) :
        try :
            text_sequence = liste_data[i]['text']
            text_sequence = text_sequence.replace('\n', ' ')
            text_result += ' ' + text_sequence
        except KeyError :
            print("Exception dans la recherche de texte")
            continue
    
    return text_result

# Crée, pour un candidat, une seule variable de texte composée du texte issu du transcript de toutes les vidéos
# J'avais un peu peur qu'on puisse overflower les strings python, mais si j'en crois ce site : https://www.tutorialspoint.com/What-is-the-max-length-of-a-Python-string
# On est laaaaaaarge
# En gros a l'issue de cette fonction, on a un très gros texte qui est le texte du transcript de toutes les vidéos des candidats, mis bout à bout
def creer_logorrhee_candidat(nom = '', dossierparent= '', liste_vidid_arecup = []) :
    # Defini le dossier ou trouver les fichiers de transcript
    nomdossier = get_dossier_candidat(nom=nom)
    dossier_candidat = dossierparent + "/" + nomdossier
    logorrhee = ''

    # Defini la liste des videos dont on veut recuperer le transcript
    for vid_id in liste_vidid_arecup :
        filename = vid_id + '.json'
        text_tampon = create_text_variable_for_transcript(nomfichier=dossier_candidat+"/"+filename)
        logorrhee += ' ' + text_tampon.lower()

    return logorrhee

### 1.2.2. Tokenization des textes générés pour faire des calculs d'occurrence des mots

In [4]:
# Analyse de texte : NLTK = Natural Language Tool Kit
import nltk
from nltk.corpus import stopwords
# Librairies pour créer les Word Cloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Tokenization et calcul NLTK sur la base de la grosse chaîne de texte générée
# Parametre permet de faire une liste d'exception custom pour chaque dataset considere
def NLTK_processing(logorrhee, liste_exception_custo = []):
    #Expression régulière pour ne conserver que les caractères alphanumérique
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    logorrhee_tokenisee = tokenizer.tokenize(logorrhee)
    # On enlève la liste des stopwords par défaut de NLTK
    logorrhee_nettoyee =[t for t in logorrhee_tokenisee if t not in stopwords.words('french')]
    logorrhee_nettoyee_custo = [t for t in logorrhee_nettoyee if t not in liste_exclusion_additionnelle]
    if len(liste_exception_custo) > 0 :
        logorrhee_nettoyee_custo = [t for t in logorrhee_nettoyee_custo if t not in liste_exception_custo]
    
    # stemmer = FrenchStemmer()
    # logorrhee_stemmee =[stemmer.stem(w) for w in logorrhee_nettoyee]
    # En fait, on ne peut pas trop stemmer pour faire des nuages de mots, sinon ces derniers sont tout pourri

    # On recolle les mots ensemble séparés par des espaces
    logorrhee_recollee = ' '.join([word for word in logorrhee_nettoyee_custo])
    return logorrhee_recollee

# Fonction qui pourrait nous permettre de refiltrer les les chaînes de textes
def additionnal_cleaning(dtf, liste = []) :
    inverse_boolean_series = ~dtf.mot.isin(liste)
    inverse_filtered_df = dtf[inverse_boolean_series]
    return inverse_filtered_df

# Calcul de fréquence de mots dans une string
def wordFrequency(string):
    # converting the string into lowercase
    string=string.lower()
    # Whenever we encounter a space, break the string
    string=string.split(" ")
    # Initializing a dictionary to store the frequency of words
    word_frequency={}
    # <a href="https://www.pythonpool.com/python-iterate-through-list/" target="_blank" rel="noreferrer noopener">Iterating</a> through the string
    for i in string:
     
    # If the word is already in the keys, increment its frequency
        if i in word_frequency:
            word_frequency[i]+=1
             
    # It means that this is the first occurence of the word
        else:
            word_frequency[i]=1
    return(word_frequency)

### 1.2.3. Dump d'un fichier .json au format attendu pour le site https://s01.810.fr 

In [5]:
import random

def dump_top_mots_candidat(dtf_synth_courant, nomfichier) :
    liste_a_dumper = []
    styles_possibles = ["400", "700", "600", '100', "bold", "normal", "500", "400", "200"]
    for index, row in dtf_synth_courant.iterrows():
        liste_a_dumper.append({"\"label\"" : "\""+ str(row["mot"]) + "\"", "\"value\"" : row["occurences"], "\"style\"" : "\""+ random.choice(styles_possibles) + "\"" })

    liste_en_geustr = str(liste_a_dumper)
    liste_en_geustr = liste_en_geustr.replace("'","")
    with open(nomfichier, "w") as text_file:
        text_file.write(liste_en_geustr)

    return

# 2. Générer les listes de vidéos sur lesquelles on aimerait travailler / candidat.

Permet d'exporter la liste des videos des candidats pour lesquelles on aimerait générer des nuages de mots.

In [6]:
# Paramètres globaux à mettre à jour si on change de fichier
dossier_source = "./temp_files/lst_vidz_for_wordcloud/"
# Modifier le nom du fichier avec le fichier souhaité
nom_fichier = "auto_candidat"

liste_candidats = ['Eric Zemmour', 'Avec vous', 'Jean-Luc Mélenchon', 'Valérie Pécresse', 'Yannick Jadot', 'Anne Hidalgo', 'Jean Lassalle']
periode_de_temps = {'datemin' : datetime.datetime(2022, 2, 1, 0, 0, 0), 'datemax' : datetime.datetime(2022, 3, 1, 0, 0, 0)}

dump_liste_videos_par_candidat(videos_candidats, liste_candidats = liste_candidats, periode_de_temps= periode_de_temps)

# 3. Créer les wordcloud issus des transcripts desdites vidéos

## 3.1. Eric Zemmour

In [7]:
indice_candidat = 0
fichier_vidz = dossier_source + 'auto_candidat_' + str(indice_candidat) + '.csv'
dossier_ts = transcripts_folder + 'EricZemmour'
df_candidat_2 = pd.read_csv(fichier_vidz)


# Voir si les vidéos considérées ont bien des transcripts
liste_videoid_sans_transcript, liste_videoid_avec_transcript = check_exhau_transcript(dtf_candidat=df_candidat_2, dossier_ts_associe = dossier_ts)
# Croiser la liste des vidéos et les fichiers de transcripts pour créer la logorrhee (gros texte de tous les mots de tous les transcripts considérés)
logoR_2 = creer_logorrhee_candidat(nom=liste_candidats[indice_candidat], dossierparent=transcripts_folder, liste_vidid_arecup = liste_videoid_avec_transcript)
# Nettoyer la logorrhee
cleanlog = NLTK_processing(logorrhee=logoR_2)

# Nettoyer un peu les scories et faire un datafram avec [mot | nb occurences]
data_dict = wordFrequency(cleanlog)
data_items = data_dict. items()
data_list = list(data_items)
df = pd. DataFrame(data_list, columns=['mot', 'occurences']) 

df = df.sort_values(by = ['occurences'], ascending=False)
df_to_export = df.head(20)

# Dump des fichiers pour https://s01.810.fr
dump_top_mots_candidat(dtf_synth_courant=df_to_export, nomfichier=folder_json+'mot_fev22-zemmour.json')
df_to_export.to_csv(folder_csv+'mot_fev22-zemmour.csv')

0/18 n'ont pas de transcript. Peut-être les sous-titres furent-ils désactivés


## 3.2. Emmanuel Macron

In [8]:
indice_candidat = 1
fichier_vidz = dossier_source + 'auto_candidat_' + str(indice_candidat) + '.csv'
dossier_ts = transcripts_folder + '/Avecvous'
df_candidat_2 = pd.read_csv(fichier_vidz)

# Voir si les vidéos considérées ont bien des transcripts
liste_videoid_sans_transcript, liste_videoid_avec_transcript = check_exhau_transcript(dtf_candidat=df_candidat_2, dossier_ts_associe = dossier_ts)
# Croiser la liste des vidéos et les fichiers de transcripts pour créer la logorrhee (gros texte de tous les mots de tous les transcripts considérés)
logoR_2 = creer_logorrhee_candidat(nom=liste_candidats[indice_candidat], dossierparent=transcripts_folder, liste_vidid_arecup = liste_videoid_avec_transcript)
# Nettoyer la logorrhee
cleanlog = NLTK_processing(logorrhee=logoR_2)

# Nettoyer un peu les scories et faire un datafram avec [mot | nb occurences]
data_dict = wordFrequency(cleanlog)
data_items = data_dict. items()
data_list = list(data_items)
df = pd. DataFrame(data_list, columns=['mot', 'occurences']) 

df = df.sort_values(by = ['occurences'], ascending=False)
df_to_export = df.head(20)

# Dump des fichiers pour https://s01.810.fr
dump_top_mots_candidat(dtf_synth_courant=df_to_export, nomfichier=folder_json+'mot_fev22-macron.json')
df_to_export.to_csv(folder_csv+'mot_fev22-macron.csv')

0/7 n'ont pas de transcript. Peut-être les sous-titres furent-ils désactivés


## 3.3. Jean-Luc Mélenchon

In [9]:
indice_candidat = 2
fichier_vidz = dossier_source + 'auto_candidat_' + str(indice_candidat) + '.csv'
dossier_ts = transcripts_folder + 'JeanLucMlenchon'
df_candidat_2 = pd.read_csv(fichier_vidz)

# Voir si les vidéos considérées ont bien des transcripts
liste_videoid_sans_transcript, liste_videoid_avec_transcript = check_exhau_transcript(dtf_candidat=df_candidat_2, dossier_ts_associe = dossier_ts)
# Croiser la liste des vidéos et les fichiers de transcripts pour créer la logorrhee (gros texte de tous les mots de tous les transcripts considérés)
logoR_2 = creer_logorrhee_candidat(nom=liste_candidats[indice_candidat], dossierparent=transcripts_folder, liste_vidid_arecup = liste_videoid_avec_transcript)
# Nettoyer la logorrhee
cleanlog = NLTK_processing(logorrhee=logoR_2)

# Nettoyer un peu les scories et faire un datafram avec [mot | nb occurences]
data_dict = wordFrequency(cleanlog)
data_items = data_dict. items()
data_list = list(data_items)
df = pd. DataFrame(data_list, columns=['mot', 'occurences']) 

df = df.sort_values(by = ['occurences'], ascending=False)
df_to_export = df.head(20)

# Dump des fichiers pour https://s01.810.fr
dump_top_mots_candidat(dtf_synth_courant=df_to_export, nomfichier=folder_json+'mot_fev22-melenchon.json')
df_to_export.to_csv(folder_csv+'mot_fev22-melenchon.csv')

0/27 n'ont pas de transcript. Peut-être les sous-titres furent-ils désactivés


## 3.4. Valérie Pécresse

In [10]:
indice_candidat = 3
fichier_vidz = dossier_source + 'auto_candidat_' + str(indice_candidat) + '.csv'
dossier_ts = transcripts_folder + 'ValriePcresse'
df_candidat_2 = pd.read_csv(fichier_vidz)

# Voir si les vidéos considérées ont bien des transcripts
liste_videoid_sans_transcript, liste_videoid_avec_transcript = check_exhau_transcript(dtf_candidat=df_candidat_2, dossier_ts_associe = dossier_ts)
# Croiser la liste des vidéos et les fichiers de transcripts pour créer la logorrhee (gros texte de tous les mots de tous les transcripts considérés)
logoR_2 = creer_logorrhee_candidat(nom=liste_candidats[indice_candidat], dossierparent=transcripts_folder, liste_vidid_arecup = liste_videoid_avec_transcript)
# Nettoyer la logorrhee
cleanlog = NLTK_processing(logorrhee=logoR_2)

# Nettoyer un peu les scories et faire un datafram avec [mot | nb occurences]
data_dict = wordFrequency(cleanlog)
data_items = data_dict. items()
data_list = list(data_items)
df = pd. DataFrame(data_list, columns=['mot', 'occurences']) 

df = df.sort_values(by = ['occurences'], ascending=False)
df_to_export = df.head(20)

# Dump des fichiers pour https://s01.810.fr
dump_top_mots_candidat(dtf_synth_courant=df_to_export, nomfichier=folder_json+'mot_fev22-pecresse.json')
df_to_export.to_csv(folder_csv+'mot_fev22-pecresse.csv')

5/24 n'ont pas de transcript. Peut-être les sous-titres furent-ils désactivés


## 3.5. Yannick Jadot

In [11]:
indice_candidat = 4
fichier_vidz = dossier_source + 'auto_candidat_' + str(indice_candidat) + '.csv'
dossier_ts = transcripts_folder + 'YannickJadot'
df_candidat_2 = pd.read_csv(fichier_vidz)

# Voir si les vidéos considérées ont bien des transcripts
liste_videoid_sans_transcript, liste_videoid_avec_transcript = check_exhau_transcript(dtf_candidat=df_candidat_2, dossier_ts_associe = dossier_ts)
# Croiser la liste des vidéos et les fichiers de transcripts pour créer la logorrhee (gros texte de tous les mots de tous les transcripts considérés)
logoR_2 = creer_logorrhee_candidat(nom=liste_candidats[indice_candidat], dossierparent=transcripts_folder, liste_vidid_arecup = liste_videoid_avec_transcript)
# Nettoyer la logorrhee
cleanlog = NLTK_processing(logorrhee=logoR_2)

# Nettoyer un peu les scories et faire un datafram avec [mot | nb occurences]
data_dict = wordFrequency(cleanlog)
data_items = data_dict. items()
data_list = list(data_items)
df = pd. DataFrame(data_list, columns=['mot', 'occurences']) 

df = df.sort_values(by = ['occurences'], ascending=False)
df_to_export = df.head(20)

# Dump des fichiers pour https://s01.810.fr
dump_top_mots_candidat(dtf_synth_courant=df_to_export, nomfichier=folder_json+'mot_fev22-jadot.json')
df_to_export.to_csv(folder_csv+'mot_fev22-jadot.csv')

1/36 n'ont pas de transcript. Peut-être les sous-titres furent-ils désactivés


## 3.6. Anne Hidalgo

In [12]:
indice_candidat = 5
fichier_vidz = dossier_source + 'auto_candidat_' + str(indice_candidat) + '.csv'
dossier_ts = transcripts_folder + 'AnneHidalgo'
df_candidat_2 = pd.read_csv(fichier_vidz)

# Voir si les vidéos considérées ont bien des transcripts
liste_videoid_sans_transcript, liste_videoid_avec_transcript = check_exhau_transcript(dtf_candidat=df_candidat_2, dossier_ts_associe = dossier_ts)
# Croiser la liste des vidéos et les fichiers de transcripts pour créer la logorrhee (gros texte de tous les mots de tous les transcripts considérés)
logoR_2 = creer_logorrhee_candidat(nom=liste_candidats[indice_candidat], dossierparent=transcripts_folder, liste_vidid_arecup = liste_videoid_avec_transcript)
# Nettoyer la logorrhee
cleanlog = NLTK_processing(logorrhee=logoR_2)

# Nettoyer un peu les scories et faire un datafram avec [mot | nb occurences]
data_dict = wordFrequency(cleanlog)
data_items = data_dict. items()
data_list = list(data_items)
df = pd. DataFrame(data_list, columns=['mot', 'occurences']) 

df = df.sort_values(by = ['occurences'], ascending=False)
df_to_export = df.head(20)

# Dump des fichiers pour https://s01.810.fr
dump_top_mots_candidat(dtf_synth_courant=df_to_export, nomfichier=folder_json+'mot_jan22-hidalgo.json')
df_to_export.to_csv(folder_csv+'mot_jan22-hidalgo.csv')

3/18 n'ont pas de transcript. Peut-être les sous-titres furent-ils désactivés


# Jean Lassalle

In [13]:
indice_candidat = 6
fichier_vidz = dossier_source + 'auto_candidat_' + str(indice_candidat) + '.csv'
dossier_ts = transcripts_folder + 'JeanLassalle'
df_candidat_2 = pd.read_csv(fichier_vidz)

# Voir si les vidéos considérées ont bien des transcripts
liste_videoid_sans_transcript, liste_videoid_avec_transcript = check_exhau_transcript(dtf_candidat=df_candidat_2, dossier_ts_associe = dossier_ts)
# Croiser la liste des vidéos et les fichiers de transcripts pour créer la logorrhee (gros texte de tous les mots de tous les transcripts considérés)
logoR_2 = creer_logorrhee_candidat(nom=liste_candidats[indice_candidat], dossierparent=transcripts_folder, liste_vidid_arecup = liste_videoid_avec_transcript)
# Nettoyer la logorrhee
cleanlog = NLTK_processing(logorrhee=logoR_2)

# Nettoyer un peu les scories et faire un datafram avec [mot | nb occurences]
data_dict = wordFrequency(cleanlog)
data_items = data_dict. items()
data_list = list(data_items)
df = pd. DataFrame(data_list, columns=['mot', 'occurences']) 

df = df.sort_values(by = ['occurences'], ascending=False)
df_to_export = df.head(20)

# Dump des fichiers pour https://s01.810.fr
dump_top_mots_candidat(dtf_synth_courant=df_to_export, nomfichier=folder_json+'mot_fev22-lassalle.json')
df_to_export.to_csv(folder_csv+'mot_fev22-lassalle.csv')

1/8 n'ont pas de transcript. Peut-être les sous-titres furent-ils désactivés
